In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib
import math

import platform
from matplotlib import font_manager, rc
matplotlib.rcParams['axes.unicode_minus'] = False # 그래프에서 음수 표현 가능하도록 설정.

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
import time, random, schedule, ctypes, os
from datetime import datetime
import requests

from pathlib import Path
import shutil

In [3]:
# 한글 폰트 지정
path = "C:/Windows/Fonts/malgun.ttf"
if platform.system() == "Windows":
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
elif platform.system()=="Darwin":
    rc('font', family='AppleGothic')
else:
    print("Unknown System")

In [8]:
import chromedriver_autoinstaller as ca

In [6]:
path = 'C:/data/Wantreez/Crawling/music/crawled_data/outlier_data/2022-05-02_2022-05-08/'
file_name = '20220502_20220508_outliers.xlsx'
file_path = path + file_name
df = pd.read_excel(file_path)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   날짜          60 non-null     object 
 1   사이트         60 non-null     object 
 2   순위          43 non-null     float64
 3   곡           60 non-null     object 
 4   가수          60 non-null     object 
 5   앨범          60 non-null     object 
 6   소속사         60 non-null     object 
 7   소속사_분류      60 non-null     object 
 8   순위변동        43 non-null     float64
 9   순위변동평균(양수)  60 non-null     float64
dtypes: float64(3), object(7)
memory usage: 4.8+ KB


In [10]:
len(df['곡'].unique())

39

In [11]:
df.shape

(39, 6)

In [12]:
# 곡 중에서 괄호, ? 제거
# ?는 파일명으로 할 수 없음
df['곡'] = df['곡'].apply(lambda x: x.split('(')[0].strip())
df['곡'] = df['곡'].apply(lambda x: x.replace('?', ''))
df.shape

(39, 6)

## 네이버 데이터랩 크롤링 함수 만들기

In [12]:
ca.install(True)
url = 'https://datalab.naver.com/keyword/trendSearch.naver'
driver = webdriver.Chrome()
driver.maximize_window()
driver.get(url)

driver.implicitly_wait(2)
time.sleep(random.uniform(1, 2))

# 작업 경로 변경
try:
    os.chdir('C:/Users/kbjung/Downloads')
except:
    pass
# 다운로드 폴더 안에 파일 수집할 폴더 만들기
scrapped_date = datetime.now().strftime('%Y%m%d')
dir_path = os.getcwd().replace('\\', '/')
os.makedirs(f'outliers_datalab_{scrapped_date}', exist_ok=True)

def datalab_crawling(df):
    
    # 주제어1 버튼 활성화
    bs(driver.page_source, 'lxml')
    do_click = driver.find_element_by_xpath('//*[@id="item_keyword1"]')
    # 검색어 입력
    do_click.send_keys('테스트')

    # 기간설정 3개월
    driver.find_element_by_xpath('//*[@id="content"]/div/div[2]/div[1]/div/form/fieldset/div/div[6]/div[1]/label[3]').click()

    # 검색버튼 클릭 첫번째랑 두번째랑 경로가 다름
    driver.find_element_by_xpath('//*[@id="content"]/div/div[2]/div[1]/div/form/fieldset/a/span').click()
    driver.implicitly_wait(2)
    time.sleep(random.uniform(1, 2))
    
    for a in range(df.shape[0]):
        # 주제어1 버튼 활성화
        bs(driver.page_source, 'lxml')
        do_click = driver.find_element_by_xpath('//*[@id="item_keyword1"]')
        # 텍스트 지우기
        do_click.clear()
        # 검색어 입력
        do_click.send_keys(df['곡'].iloc[a])

        # 주제어1 서브키워드 지우기
        do_click = driver.find_element_by_xpath('//*[@id="item_sub_keyword1_1"]')
        do_click.clear()
        driver.implicitly_wait(2)

        # 기간설정 3개월
        driver.find_element_by_xpath('//*[@id="content"]/div[1]/div[1]/div[2]/div[2]/form/fieldset/div/div[6]/div[1]/label[3]').click()

        # 검색버튼 클릭     
        driver.find_element_by_xpath('//*[@id="content"]/div[1]/div[1]/div[2]/div[2]/form/fieldset/a').click()
        driver.implicitly_wait(2)
        time.sleep(random.uniform(1, 2))

        # 엑셀 다운로드 버튼 클릭
        driver.find_element_by_xpath('//*[@id="content"]/div[1]/div[1]/div[1]/div/div/div/div/div/div[1]/div[4]/a').click()
        driver.implicitly_wait(2)
        time.sleep(random.uniform(4, 5))
        
        ## 다운로드한 파일 설정
        # 작업 경로 변경
        try:
            os.chdir('C:/Users/kbjung/Downloads')
        except:
            pass
        # 폴더 만들기
        dir_path = os.getcwd().replace('\\', '/')

        # 똑같은 이름의 파일이 있으면 FileExistsError남
        try:
            # 파일 이름 바꾸기 datalab -> target_df의 곡 이름
            old_name = os.path.join(dir_path, 'datalab.xlsx')
            new_name = os.path.join(dir_path, df['곡'].iloc[a]+'.xlsx')
            os.rename(old_name, new_name)
        except:
            pass
        
        # 이름 바꾼 파일 outliers_datalab 폴더에 넣어주기
        file_source = os.path.join(dir_path, f"{df['곡'].iloc[a]+'.xlsx'}")
        file_destination = os.path.join(dir_path, f'outliers_datalab_{scrapped_date}')
        try:
            shutil.move(file_source, file_destination)
        except:
            pass
        
        time.sleep(random.uniform(10, 11))
        
    # 다운로드 폴더 파일 엑셀 모여있는 폴더로 옮기기
    file_source = os.path.join(dir_path, f"outliers_datalab_{scrapped_date}")
    file_destination = path
    try:
        shutil.move(file_source, file_destination)
    except:
        pass
        
datalab_crawling(df)

C:\Users\kbjung\AppData\Local\Temp/ipykernel_11880/1618147306.py:24: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  do_click = driver.find_element_by_xpath('//*[@id="item_keyword1"]')
C:\Users\kbjung\AppData\Local\Temp/ipykernel_11880/1618147306.py:29: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="content"]/div/div[2]/div[1]/div/form/fieldset/div/div[6]/div[1]/label[3]').click()
C:\Users\kbjung\AppData\Local\Temp/ipykernel_11880/1618147306.py:32: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="content"]/div/div[2]/div[1]/div/form/fieldset/a/span').click()
C:\Users\kbjung\AppData\Local\Temp/ipykernel_11880/1618147306.py:39: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(